<a href="https://colab.research.google.com/github/benza613/Sentiment-Analysis-using-Deep-Learning/blob/master/notebooks/model_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd /content/drive/My\ Drive/SNLP\ Project

In [0]:
# load your choice of dataset here . Specify paths as folder_datestring/file_datestring.zip
Train_ZipCSVFileName = 'amz_all_electronics/Data_Balanced_2000_Apr-03-2020_06-46/Train_2000_Apr-03-2020_06-46.zip'
Test_ZipCSVFileName = 'amz_all_electronics/Data_Balanced_2000_Apr-03-2020_06-46/Test_2000_Apr-03-2020_06-46.zip'
import pandas as pd
import numpy as np


df_train = pd.read_csv(Train_ZipCSVFileName)
df_train.info()

df_test = pd.read_csv(Test_ZipCSVFileName)
df_test.info()

In [0]:
# Get names of indexes for which column overall has value 3
index_neutrals_train = df_train[ df_train['overall'] == 3 ].index
index_neutrals_test = df_test[ df_test['overall'] == 3 ].index
 
# Delete these row indexes from dataFrame
df_train.drop(index_neutrals_train , inplace=True)
df_test.drop(index_neutrals_test , inplace=True)

df_train.loc[(df_train.overall == 1),'overall']= 1
df_train.loc[(df_train.overall == 2),'overall']= 1
df_train.loc[(df_train.overall == 4),'overall']= 5
df_train.loc[(df_train.overall == 5),'overall']= 5

df_test.loc[(df_test.overall == 1),'overall']= 1
df_test.loc[(df_test.overall == 2),'overall']= 1
df_test.loc[(df_test.overall == 4),'overall']= 5
df_test.loc[(df_test.overall == 5),'overall']= 5

df_train['reviewText_len'].describe()
# Since the mean average review size is around 145 chars and max is 400, I can safely set the max [summary + review] Text Limit to 400 

In [0]:
df_train['overall'].describe()
df_test['overall'].describe()

In [0]:
# The maximum number of words to be used. (most frequent)
MAX_VOCAB_SIZE = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 200

import uuid
folderGUID = uuid.uuid4().hex

# stupid shell way of converting variable to string 
!mkdir "ModelResults/v3_$folderGUID"

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer(num_words= MAX_VOCAB_SIZE, filters='#$%&()*+<=>@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(df_train['summary'].values + ' DELIM '+ df_train['reviewText'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [0]:
X = tokenizer.texts_to_sequences(df_train['summary'].values + ' DELIM '+ df_train['reviewText'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


In [0]:
Y = pd.get_dummies(df_train['overall']).values
print('Shape of label tensor:', Y.shape)

In [0]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

embedding_dim = 100

model = Sequential()
model.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM,input_length=X.shape[1]))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',f1_m,precision_m, recall_m])

#model.summary()
epochs = 3

history = model.fit(X,Y,epochs=epochs);

In [0]:
# Save the model to Local Disk 
# https://stackoverflow.com/questions/45424683/how-to-continue-training-for-a-saved-and-then-loaded-keras-model

filePath = "ModelResults/v3_"+str(folderGUID)+"/model.h5"
model.save(filePath)
print("Saved model to disk : "+ str(folderGUID))

In [0]:
# Run additional training if necessary & remember to resave it 

# Load the model
# model = load_model(filePath)

# Train more on the loaded model
#history = model.fit(X, Y, epochs=epochs)

In [0]:
X_test = tokenizer.texts_to_sequences(df_test['summary'].values + ' DELIM '+df_test['reviewText'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Y_Test = pd.get_dummies(df_test['overall']).values
print('Shape of label tensor:', Y_Test.shape)

loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, Y_Test, verbose=0)

print("loss",loss)
print("accuracy",accuracy)
print("f1 score",f1_score)
print("precision",precision)
print("recall",recall)

In [0]:
from matplotlib import *
import matplotlib.pyplot as plt
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
#plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
#plt.plot(history.history['val_acc'], label='test')
plt.legend()
plt.show();

In [0]:
# Test for single review 
new_review = ['I am a victim of identity theft and someone stole my identity and personal information to open up a Visa credit card account with Bank of America. The following Bank of America Visa credit card account do not belong to me : XXXX.']
seq = tokenizer.texts_to_sequences(new_review)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)


print(pred)
print(np.argmax(pred))